# Deep Learning Aplicado a la Detección Automática de Valvulopatías Usando Sonidos Cardíacos
John Jairo Gelpud Chachinoy<sup>1</sup>, Silvia Gabriela Castillo Pastuzan<sup>2</sup>

Director: PhD. Wilson O. Achicanoy Martínez<sup>3</sup>

Coasesor: PhD(c). Mario Fernando Jojoa<sup>4</sup>

<sup>1</sup>Departamento de Electrónica, Universidad de Nariño, Pasto, e-mail: johnjairog@udenar.edu.co

<sup>2</sup>Departamento de Electrónica, Universidad de Nariño, Pasto, e-mail:gabrielacast@udenar.edu.co

<sup>3</sup>Departamento de Electrónica, Universidad de Nariño, Pasto, e-mail:wilachic@udenar.edu.co

<sup>4</sup>Universidad de Deusto, España, e-mail:mariojojoa@deusto.es


# Introducción
El presente código hace parte de los documentos desarrollados como parte del trabajo de tesis de grado para optar por el título de ingeniero electrónico en la Universidad de Nariño. ...

## Código de Inicialización

### Importar Librerías

In [ ]:
from __future__ import print_function, division

import torch
import torch.nn as nn
import torch.backends.cudnn as cudnn
import numpy as np
import torchvision
from torchvision import datasets, models, transforms
import matplotlib.pyplot as plt
import time
import os
import copy
import zipfile
import io
import pandas as pd
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.model_selection import GridSearchCV, StratifiedKFold
from sklearn.metrics import accuracy_score, recall_score, precision_score
import scikitplot as skplt

plt.style.use('seaborn-white')
plt.rcParams.update({'font.size':18})

cudnn.benchmark = True
plt.ion()   # interactive mode

### Cargar Datos

In [ ]:
# Si se está ejecutando en Google Colab definir colab=True
colab = False
# Si se está ejecutando localmente especificar el directorio en dataset_path
dataset_path = 'D:/UDENAR/Electronic Engineering/Trabajo de Grado/Results/Teager/Classification/CardioDeep2/Datasets/'
features_path = ''

if colab:
    dataset_path = 'Datasets/'
    from google.colab import files
    uploaded = files.upload()
    data = zipfile.ZipFile(io.BytesIO(uploaded['Datasets.zip']), 'r')
    data.extractall()
else:
    print('Ejecutando en sistema local')

In [ ]:
# Normalización de las imágenes para la etapa de entrenamiento y validación
data_transforms = {
    'train':transforms.Compose([
        transforms.Resize(224),
        transforms.CenterCrop(224),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ]),
    'val':transforms.Compose([
        transforms.Resize(224),
        transforms.CenterCrop(224),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ]),
    'test':transforms.Compose([
        transforms.Resize(224),
        transforms.CenterCrop(224),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ]),
}

In [ ]:
# Crea los datasets y dataloaders con un batch de 4 imágenes, y aplica las transformaciones
image_datasets = {x: datasets.ImageFolder(dataset_path+x, data_transforms[x])
                      for x in ['train', 'val', 'test']}
dataloaders = {x: torch.utils.data.DataLoader(image_datasets[x], batch_size=1, shuffle=False,
                                              num_workers=2) for x in ['train', 'val', 'test']}
dataset_sizes = {x: len(image_datasets[x]) for x in ['train', 'val', 'test']}
class_names = image_datasets['train'].classes # nombres de las categorias
# Selecciona la GPU o CPU como dispositivo para realizar las operaciones y
# almacenar variables
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')

## Feature Extraction

### Funciones

In [ ]:
def feature_extraction(model, dataloaders, dataset_sizes, num_ftrs):
    train_ftrs = torch.empty((dataset_sizes['train'], num_ftrs+1), device=device)
    val_ftrs = torch.empty((dataset_sizes['val'], num_ftrs+1), device=device)
    test_ftrs = torch.empty((dataset_sizes['test'], num_ftrs+1), device=device)
    
    for dataset, array in zip(['train', 'val', 'test'], [train_ftrs, val_ftrs, test_ftrs]):
        
        for i, data in zip(range(dataset_sizes[dataset]), dataloaders[dataset]):
            inputs = data[0]
            labels = data[1]
            inputs = inputs.to(device)
            labels = labels.to(device)
            outputs = model(inputs)
            array[i, :-1] = outputs.squeeze()
            array[i, -1] = labels
    
    train_ftrs = train_ftrs.cpu().detach().numpy()
    val_ftrs = val_ftrs.cpu().detach().numpy()
    test_ftrs = test_ftrs.cpu().detach().numpy()
    
    return train_ftrs, val_ftrs, test_ftrs

In [ ]:
def save_features(train_ftrs, val_ftrs, test_ftrs, model, path):
    columns = ['ftr_'+str(x) for x in range(train_ftrs.shape[1]-1)]
    columns.append('label')
    
    train_dataset = pd.DataFrame(train_ftrs, columns=columns)
    val_dataset = pd.DataFrame(val_ftrs, columns=columns)
    test_dataset = pd.DataFrame(test_ftrs, columns=columns)
    
    for phase, dataframe in zip(['train', 'val', 'test'], [train_dataset, val_dataset, test_dataset]):
        dataframe.to_excel(path+model+'/'+phase+'.xlsx')
    
    return train_dataset, val_dataset, test_dataset

### ConvNets as features extractors

#### ResNet152

In [ ]:
res_model = models.resnet152(pretrained=True)
res_num_ftrs = res_model.fc.in_features

modules = list(res_model.children())[:-1]
fe_res_model = nn.Sequential(*modules)

for param in res_model.parameters():
    param.requires_grad = False

fe_res_model.eval()
fe_res_model.to(device)

In [ ]:
res_train_ftrs, res_val_ftrs, res_test_ftrs = feature_extraction(fe_res_model, dataloaders,
                                                                 dataset_sizes, res_num_ftrs)

In [ ]:
res_train_df, res_val_df, res_test_df = save_features(res_train_ftrs, res_val_ftrs,
                                                      res_test_ftrs, 'ResNet152',
                                                      features_path)

#### VGG16

In [ ]:
vgg_model = models.vgg16(pretrained=True)
vgg_num_ftrs = list(vgg_model.classifier.children())[-1].in_features

fe_vgg_model = vgg_model
fe_vgg_model.classifier = fe_vgg_model.classifier[:-1]

for param in fe_vgg_model.parameters():
    param.requires_grad = False
    
fe_vgg_model.eval()
fe_vgg_model.to(device)

In [ ]:
vgg_train_ftrs, vgg_val_ftrs, vgg_test_ftrs = feature_extraction(fe_vgg_model, dataloaders,
                                                                 dataset_sizes, vgg_num_ftrs)

In [ ]:
vgg_train_df, vgg_val_df, vgg_test_df = save_features(vgg_train_ftrs, vgg_val_ftrs,
                                                      vgg_test_ftrs, 'VGG16',
                                                      features_path)